In [1]:
from scripts.grid_generators import *
from scripts.neural_networks import *
from models.MavelliPURE import *


import numpy as np
import pandas as pd
import os
from scipy.integrate import odeint
import matplotlib.pyplot as plt


In [2]:
Grid_Path = "./datasets/grids/"
Plot_Path = "./Plots/"


In [3]:
TargetSpecies = {
                     "NTP" : {"Look_Up": "NTP", "initial_condition_vector_index" : 0, "max_conc_mM" : 1500},
                     "tRNA" : {"Look_Up" : "T", "initial_condition_vector_index" : 6, "max_conc_mM" : 1.9},
                     "Amino Acids" : {"Look_Up" : "A", "initial_condition_vector_index" : 7, "max_conc_mM" : 300},
                     "Creatine_Phosphate" : {"Look_Up" : "CP", "initial_condition_vector_index" : 8, "max_conc_mM" : 20000},
                     "TL Enzymes" : {"Look_Up" : "CTL", "initial_condition_vector_index" : 10, "max_conc_mM" : 2.2}
}

# iterate over TargetSpecies and store that max concs in a list which is converted to a np.array
max_concs_array = []
for key in TargetSpecies:
    max_concs_array.append(TargetSpecies[key]["max_conc_mM"])
    
max_concs_array = np.array(max_concs_array)

In [4]:
PermissiblePercentagesOfMaxConcs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
grid_size = 30
NumOfTargetSpecies = len(TargetSpecies)

In [5]:
initialgrid, array_to_avoid = generate_initial_grid(grid_size, max_concs_array, PermissiblePercentagesOfMaxConcs, NumOfTargetSpecies)

initialgrid_df = pd.DataFrame(initialgrid, columns=TargetSpecies.keys())

initialgrid_df.to_csv(Grid_Path+"initial_grid_mM.csv", index=None)

# iterate over initial grid

In [6]:

keysVar = ['NTP','NXP','nt','Ppi','ATr','a','T','A','CP','C','CTL']
valuesVar = [1500,0,0,0,0,0,1.9,300,20000,0,2.2]

initial_concs_dict = dict(zip(keysVar, valuesVar))


In [7]:
TMAX = 6*60*60 # sec
NSTEPS = TMAX

In [8]:

endpoint_protein_concentrations = []

for composition in initialgrid:
    
    original_concentrations = list(initial_concs_dict.values())
    updated_concentrations = original_concentrations.copy()
    
    for new_conc, key in zip(composition, TargetSpecies):
        
        
        index = TargetSpecies[key]["initial_condition_vector_index"]
        
        updated_concentrations[index] = new_conc
        
    
    sol, time = solvePURE(TMAX, NSTEPS, updated_concentrations)
    
    # last time point and just get the polymerised protein at index 5
    endpoint_protein_concentrations.append(sol[-1,:][5])
    
initialgrid_df["Predicted Final Protein"] = endpoint_protein_concentrations

initialgrid_df

,NTP,tRNA,Amino Acids,Creatine_Phosphate,TL Enzymes,Predicted Final Protein
0,1500.0,1.90,300.0,20000.0,2.20,105.972956
1,150.0,1.90,300.0,20000.0,2.20,105.443377
2,1500.0,0.19,300.0,20000.0,2.20,14.578242
3,1500.0,1.90,30.0,20000.0,2.20,30.000000
4,1500.0,1.90,300.0,2000.0,2.20,104.719585
5,1500.0,1.90,300.0,20000.0,0.22,10.606162
6,150.0,0.19,30.0,2000.0,0.22,1.420905
7,1500.0,0.19,30.0,2000.0,0.22,1.443418
8,150.0,1.90,30.0,2000.0,0.22,10.325757
9,150.0,0.19,300.0,2000.0,0.22,1.421121


# Train Neural Networks

In [9]:
initialgrid_df

,NTP,tRNA,Amino Acids,Creatine_Phosphate,TL Enzymes,Predicted Final Protein
0,1500.0,1.90,300.0,20000.0,2.20,105.972956
1,150.0,1.90,300.0,20000.0,2.20,105.443377
2,1500.0,0.19,300.0,20000.0,2.20,14.578242
3,1500.0,1.90,30.0,20000.0,2.20,30.000000
4,1500.0,1.90,300.0,2000.0,2.20,104.719585
5,1500.0,1.90,300.0,20000.0,0.22,10.606162
6,150.0,0.19,30.0,2000.0,0.22,1.420905
7,1500.0,0.19,30.0,2000.0,0.22,1.443418
8,150.0,1.90,30.0,2000.0,0.22,10.325757
9,150.0,0.19,300.0,2000.0,0.22,1.421121


In [10]:

# Select the input data using the TargetSpecies.keys

TargetSpeciesKeys = list(TargetSpecies.keys())

x_train = initialgrid_df[TargetSpeciesKeys].values


# produces np array of 1D
y_train = initialgrid_df["Predicted Final Protein"].values

y_train  = np.expand_dims(y_train, axis=1)
print(y_train)


[[105.97295608]
 [105.44337679]
 [ 14.57824191]
 [ 30.        ]
 [104.71958494]
 [ 10.6061623 ]
 [  1.42090454]
 [  1.44341845]
 [ 10.32575669]
 [  1.42112081]
 [  1.45178927]
 [ 14.16376891]
 [ 43.66399001]
 [ 19.66159128]
 [ 88.52094252]
 [ 31.72406367]
 [  6.15722909]
 [ 21.67458194]
 [ 10.59876343]
 [ 14.47243886]
 [  5.52508029]
 [  5.590238  ]
 [  8.11304181]
 [ 14.47736522]
 [ 24.13159135]
 [ 25.82054944]
 [ 36.03932061]
 [ 18.74271557]
 [  4.37475104]
 [ 71.80507869]]


In [11]:
input_nodes = len(TargetSpecies)
num_output_nodes = 1

In [12]:
model = define_model(input_nodes, num_output_nodes)
print(type(model))


<class 'keras.engine.sequential.Sequential'>


In [13]:
x_train


array([[1.50e+03, 1.90e+00, 3.00e+02, 2.00e+04, 2.20e+00],
       [1.50e+02, 1.90e+00, 3.00e+02, 2.00e+04, 2.20e+00],
       [1.50e+03, 1.90e-01, 3.00e+02, 2.00e+04, 2.20e+00],
       [1.50e+03, 1.90e+00, 3.00e+01, 2.00e+04, 2.20e+00],
       [1.50e+03, 1.90e+00, 3.00e+02, 2.00e+03, 2.20e+00],
       [1.50e+03, 1.90e+00, 3.00e+02, 2.00e+04, 2.20e-01],
       [1.50e+02, 1.90e-01, 3.00e+01, 2.00e+03, 2.20e-01],
       [1.50e+03, 1.90e-01, 3.00e+01, 2.00e+03, 2.20e-01],
       [1.50e+02, 1.90e+00, 3.00e+01, 2.00e+03, 2.20e-01],
       [1.50e+02, 1.90e-01, 3.00e+02, 2.00e+03, 2.20e-01],
       [1.50e+02, 1.90e-01, 3.00e+01, 2.00e+04, 2.20e-01],
       [1.50e+02, 1.90e-01, 3.00e+01, 2.00e+03, 2.20e+00],
       [7.50e+02, 9.50e-01, 2.40e+02, 1.60e+04, 1.54e+00],
       [1.05e+03, 1.71e+00, 2.10e+02, 1.40e+04, 4.40e-01],
       [1.50e+03, 1.71e+00, 3.00e+02, 1.80e+04, 1.98e+00],
       [1.20e+03, 1.90e+00, 2.10e+02, 8.00e+03, 6.60e-01],
       [9.00e+02, 9.50e-01, 2.40e+02, 2.00e+03, 2.20e-01

In [23]:
model.compile(optimizer='rmsprop',
              loss=keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])


model.fit(x_train, y_train, epochs=10)

print(model.summary())

Epoch 1/10
1/1 [==============================] - 0s 499ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 16ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 13ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 11ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 9/10
1/1 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 10/10
1/1 [==============================] - 0s 8ms/step - loss: nan - accuracy: 0.0000e+00
Model: "sequential"
__

In [15]:
random_grid_size = 1000
simulate_input, array_to_avoid = generate_random_grid(array_to_avoid, max_concs_array, random_grid_size, NumOfTargetSpecies, PermissiblePercentagesOfMaxConcs)

print(simulate_input.shape)
print(type(simulate_input))
print(simulate_input[0])

(1000, 5)
<class 'numpy.ndarray'>
[1.35e+03 1.33e+00 3.00e+01 1.40e+04 6.60e-01]


In [16]:
predictions = model.predict(simulate_input)
predictions = predictions.reshape(-1)


32/32 [==============================] - 0s 2ms/step


In [17]:
print(predictions[0])
print(type(predictions[0]))
print(predictions)

nan
<class 'numpy.float32'>
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan na

In [18]:
simulate_input_preds = pd.DataFrame(simulate_input, columns = TargetSpeciesKeys)
simulate_input_preds["Predicted Final Protein"] = predictions
#simulate_input_preds["Predicted Final Protein"] = simulate_input_preds["Predicted Final Protein"].astype(np.float64)

In [19]:
#simulate_input_preds = simulate_input_preds.apply(np.nan_to_num)

print((type(simulate_input_preds["Predicted Final Protein"][0])))

print(type(simulate_input_preds["Creatine_Phosphate"][0]))
print(simulate_input_preds)

print(simulate_input_preds["Creatine_Phosphate"].dtype())

<class 'numpy.float32'>
<class 'numpy.float64'>
        NTP  tRNA  Amino Acids  Creatine_Phosphate  TL Enzymes  \
0    1350.0  1.33         30.0             14000.0        0.66   
1    1350.0  0.95        270.0              2000.0        1.76   
2     450.0  0.76         60.0              4000.0        0.22   
3    1200.0  0.95        300.0             16000.0        1.76   
4     450.0  1.14        270.0             10000.0        2.20   
..      ...   ...          ...                 ...         ...   
995   600.0  1.33         30.0             20000.0        1.54   
996  1350.0  1.90        150.0             20000.0        1.76   
997  1500.0  0.95         60.0             12000.0        1.98   
998  1500.0  0.38         90.0              8000.0        0.22   
999  1350.0  0.76        210.0             20000.0        1.54   

     Predicted Final Protein  
0                        NaN  
1                        NaN  
2                        NaN  
3                        NaN  
4   

TypeError: 'numpy.dtype[float64]' object is not callable